In [3]:
import numpy as np
import pandas as pd
from random import choice

person = ['Sujan', 'Aravind', 'Remo', 
          'Lakshmi', 'Bhaskar', 'Priya',
          'Surya', 'Akash', 'Trisul',
          'Anji', 'Nani', 'Abhay'
         ]

n= len(person)
random = {'Height': np.random.normal(160, 3, n),
        'Weight': np.random.normal(78, 2.5, n),
        'Obese' :  np.random.choice(['yes','no'],n)
       }
df = pd.DataFrame(data=random, index=person)
df.to_excel (r'C:\codes\Books\NA19B033 MA5710 Assignment 8\Obesity.xlsx', index = True, header=True)

In [4]:
training_data=np.array(df.sort_values(by=['Weight']))
training_data

array([[162.01882564443454, 73.94319409012766, 'yes'],
       [160.80076483481125, 76.36714368441032, 'yes'],
       [157.2274224935155, 76.77329592972649, 'yes'],
       [160.8815974849233, 77.0695606022087, 'no'],
       [160.5563708027974, 79.41333284649737, 'yes'],
       [158.9689308329849, 79.46393787997525, 'yes'],
       [158.09184307724973, 79.57715888584768, 'no'],
       [156.96064503656407, 79.9736168924075, 'no'],
       [159.848669809707, 80.78429327998444, 'yes'],
       [164.71602390885198, 81.21684429411748, 'no'],
       [159.18513807446362, 81.2662272455457, 'yes'],
       [159.84897768882385, 84.19467023093013, 'no']], dtype=object)

In [5]:
header=["Height","Weight","Obese"]
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

class Question:
    
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val <= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = "<="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))


In [6]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
        return impurity
    
def gini_split(left, right,current_uncertainty):
    
    n1 = float(len(left)) / (len(left) + len(right))
    n2 = float(len(right)) / (len(left) + len(right))
    
    return  n1 * gini(left) + n2 * gini(right)


def test_split(index, value, training_data):
    left, right = list(), list()
    for row in training_data:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right        
def mid_split_points(rows,col):
    mylist=[]
    for i in range(len(rows)):
        mylist.append(rows[i][col])
    mylist.sort()
    mylist=list(set(mylist))
    length=len(mylist)
    mid_points=[]
    for i in range(length-1):
        mid_points.append((mylist[i]+mylist[i+1])/2)    
    return mid_points

def find_best_split(rows, col):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 100000  # keep track of the best information gain
    best_question = None  
    current_uncertainty = gini(rows)
    
    if is_numeric(rows[0][col]):
        values = set(mid_split_points(rows,col))
    else:    
        values = set([row[col] for row in rows])  # unique values in the column
        
    for val in values:  # for each value
    
        question = Question(col, val)
    
        # try splitting the dataset
        true_rows, false_rows = partition(rows, question)
    
        # Skip this split if it doesn't divide the
        # dataset.
        if len(true_rows) == 0 or len(false_rows) == 0:
            continue
    
        # Calculate the information gain from this split
        gain = gini_split(true_rows, false_rows,current_uncertainty)
    
    
        if gain <= best_gain:
            best_gain, best_question = gain, question

    return best_gain, best_question

def gini_index(groups,classes):
    n_instances = float(sum([len(group) for group in groups]))

    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0.0:
            continue
        score = 0.0

        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p

        gini += (1.0 - score) * (size / n_instances)
    return gini

def get_split(training_data):
    class_values = list(set(row[-1] for row in training_data))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(training_data[0])-1):
        for row in training_data:
            groups = test_split(index, row[index], training_data)
            gini = gini_index(groups, class_values)
            print('column%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


split = get_split(training_data)
print('Split: [column%d < %.3f]' % ((split['index']+1), split['value']))
        
    
    

column1 < 162.019 Gini=0.483
column1 < 160.801 Gini=0.479
column1 < 157.227 Gini=0.424
column1 < 160.882 Gini=0.444
column1 < 160.556 Gini=0.486
column1 < 158.969 Gini=0.444
column1 < 158.092 Gini=0.483
column1 < 156.961 Gini=0.486
column1 < 159.849 Gini=0.486
column1 < 164.716 Gini=0.424
column1 < 159.185 Gini=0.479
column1 < 159.849 Gini=0.472
column2 < 73.943 Gini=0.486
column2 < 76.367 Gini=0.455
column2 < 76.773 Gini=0.417
column2 < 77.070 Gini=0.370
column2 < 79.413 Gini=0.458
column2 < 79.464 Gini=0.419
column2 < 79.577 Gini=0.361
column2 < 79.974 Gini=0.438
column2 < 80.784 Gini=0.479
column2 < 81.217 Gini=0.444
column2 < 81.266 Gini=0.483
column2 < 84.195 Gini=0.424
Split: [column2 < 79.577]


In [7]:
class Leaf:
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)


class Decision_Node:
    
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


def build_tree(rows, start_col, finish_col):
    
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gini_index = gini(rows)
    
    if(gini_index == 0):
        return Leaf(rows)
    
    
    gain, question = find_best_split(rows, start_col)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    #if gain == 0:
    #    return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows, start_col + 1, finish_col)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows, start_col + 1, finish_col)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)


def print_tree(node, spacing=""):
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


def classify(row, node):
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

def print_leaf(counts):
    
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

if __name__ == '__main__':

    my_tree = build_tree(training_data,0,1)

    print_tree(my_tree)


Is Height <= 157.0940337650398?
--> True:
  Predict {'no': 1}
--> False:
  Is Weight <= 79.52054838291147?
  --> True:
    Is Obese == no?
    --> True:
      Predict {'no': 1}
    --> False:
      Predict {'yes': 5}
  --> False:
    Is Obese == no?
    --> True:
      Predict {'no': 3}
    --> False:
      Predict {'yes': 2}
